In [2]:
from pathlib import Path

In [20]:
results = []

for file in Path("/tmp").glob("ggpt*.txt"):
    current = {}
    with open(file, "r") as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            if line.startswith("🔵 Loading object masks"):
                if current != {}:
                    results.append(current)
                current = {}
            elif line.startswith("📝 Model Output:"):
                text_start = line.find('"')
                text_end = line.rfind('"')
                current["text"] = line[text_start + 1:text_end]
            elif line.startswith("🎯 Randomly selected object:"):
                object_start = line.find(':')
                id_start = line.find('(')
                id_end = line.rfind(')')
                current["object"] = line[object_start + 1:id_start].strip()
                current["id"] = line[id_start + 1:id_end]
            elif line.startswith("✅ Saved visualization to"):
                output_filename = line[line.find("to") + 3:].strip()
                current["output_filename"] = output_filename
                assert "_0/viz_random_object/result.png" in output_filename
                tmp = output_filename.replace("_0/viz_random_object/result.png", "")
                parts = tmp.split("/")
                current["orginal_id"] = parts[-1]
            else:
                continue

In [21]:
results, len(results)

([{'object': 'pillow_20',
   'id': 'ID 62',
   'text': "Look at that chair over there, it's so comfortable.",
   'output_filename': '/data/shared/mm_conv/screenshots_1.6/hsi_3_0715_227_004_main_0/viz_random_object/result.png',
   'orginal_id': 'hsi_3_0715_227_004_main'},
  {'object': 'Floor',
   'id': 'ID 14',
   'text': 'Look at the floor, this is where we will dance tonight.',
   'output_filename': '/data/shared/mm_conv/screenshots_1.6/hsi_7_0719_211_002_main_0/viz_random_object/result.png',
   'orginal_id': 'hsi_7_0719_211_002_main'},
  {'object': 'Vase_58f3a922',
   'id': 'ID 39',
   'text': "Look at that vase on the shelf, it's so beautiful.",
   'output_filename': '/data/shared/mm_conv/screenshots_1.6/hsi_7_0719_209_001_main_0/viz_random_object/result.png',
   'orginal_id': 'hsi_7_0719_209_001_main'},
  {'object': 'Door',
   'id': 'ID 17',
   'text': "I'm going to grab that book from the shelf over there.",
   'output_filename': '/data/shared/mm_conv/screenshots_1.6/hsi_6_0718_21

In [22]:
import json
with open("/tmp/ggpt2.json", "w") as f:
    json.dump(results, f, indent=4)

In [23]:
numbered = []
for idx, res in enumerate(results):
    parts = res["orginal_id"].split("_")
    numbered.append({
        "person": parts[1],
        "room": parts[3],
        "filename": res["orginal_id"],
        "id": f"{res['orginal_id']}__ggpt__{idx}",
        "topic": res["object"],
        "snippet": res["text"]
    })

In [24]:
with open("/tmp/ggpt_numbered2.json", "w") as f:
    json.dump(numbered, f, indent=4)